In [1]:
from stegosaurus.acrostic.params import AcrosticGeneratorInputs, AcrosticGenerationRoundResult
from stegosaurus.acrostic.chains import full_chain, original_text_summarizer_chain, multi_acrostic_generator_prompt, generator_chain
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
import textwrap
from pprint import pprint

%load_ext autoreload
%autoreload 2

In [3]:
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

precision_principle = ConstitutionalPrinciple(
    name="reflects_current_idea",
    critique_request="The list of proposed new sentences should exactly reflect the current idea we want to express",
    revision_request="Rewrite the model's output to exactly reflect the current idea we want to express",
)

In [16]:
critique_prompt = PromptTemplate(
    input_variables=["current_idea", "current_letter", "generated_sentence_options"],
    template="""
I asked an AI named AGA to generate a list of sentences that are meant to express this idea:
{current_idea}

AGA is also told that the first letter of each sentence must be {current_letter}.

Here is the list of sentences AGA generated:
{generated_sentence_options}

For each sentence, please determine whether it expresses the idea I gave AGA and whether it starts with the letter {current_letter}. If it does, please write "yes". If it does not, please write "no".
"""
    )

revision_prompt = PromptTemplate(
    input_variables=["current_idea", "current_letter", "generated_sentence_options", "generated_sentence_options_critique"],
    template="""
I asked an AI named AGA to generate a list of sentences that are meant to express this idea:
{current_idea}

AGA is also told that the first letter of each sentence must be {current_letter}.

Here is the list of sentences AGA generated:
{generated_sentence_options}

Here is the list of sentences AGA generated, along with your critique:
{generated_sentence_options_critique}

Wherever necessary, please revise each sentence to exactly reflect the current idea we want to express and to start with the letter {current_letter}.
"""
)

In [ ]:
critique_chain = LLMChain(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0.7), 
    prompt=critique_prompt,
    output_key="generated_sentence_options_critique")
revision_chain= LLMChain(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
    prompt=revision_prompt,
    output_key="revised_generated_sentence_options")

# test

In [7]:
initial_inputs = AcrosticGeneratorInputs(
    original_text = """
I know a yellow labrador retriever named Lily.
She is a little over 2 years old, which is pretty much grown up.
She likes to play fetch a lot and it's a good way to tire her out.
She doesn't like to cuddle much, but as she gets older she likes it a bit more.
    """,
    acrostic_phrase = "LILY",
    rewritten_text = "",
    acrostic_letter_index = 0,
    n_attempts=20,
)
results = []
verbose = True
inputs = initial_inputs.copy()

In [8]:
current_round_number = 0
n_regenerations_for_current_round = 0

In [11]:
output = full_chain(inputs=inputs.dict())
current_round_result = AcrosticGenerationRoundResult(
    round_number=current_round_number,
    inputs=inputs.copy(),
    output=output,
)
results.append(current_round_result)

if output['next_action'] == 'accept sentence':
    
    selection_idx = output['evaluator_selection'] - 1
    accepted_sentence = output['generated_sentence_options_list'][selection_idx] + ' '
    inputs.rewritten_text += accepted_sentence
    print(f'Accepted sentence:\n{output["evaluator_selection"]}. {accepted_sentence}')
    inputs.acrostic_letter_index += 1
    n_regenerations_for_current_round = 0
else:
    print(f'{output["next_action"]}')
    n_regenerations_for_current_round += 1
current_round_number += 1

print(f"""
REWRITTEN TEXT
{textwrap.fill(output['rewritten_text'])}
      
SUMMARY
{output['original_text_summary'].strip()}

CONTEXT
{textwrap.fill(output['original_text_context'])}

CURRENT SENTENCE
{textwrap.fill(output['current_original_sentence'].strip())}

OPTIONS
{output['clean_options']}
""")


inputs

Accepted sentence:
2. Lily's favorite activity is playing fetch, which helps wear her out. 

REWRITTEN TEXT
Lily is the name of the yellow labrador retriever I know. In just a
little over 2 years, Lily has become a fully grown adult.
      
SUMMARY
- The author knows a yellow labrador retriever named Lily.
- Lily is a little over 2 years old and considered an adult.
- Playing fetch is a favorite activity for Lily and helps tire her out.
- Lily isn't big on cuddling, but she's starting to enjoy it more as she gets older.

CONTEXT
  The purpose of this text is to provide information about a yellow
labrador retriever named Lily. The author is likely a pet owner or dog
enthusiast sharing details about their pet. They could be speaking to
a friend or family member who is also interested in dogs or
considering getting one. The style of the text is casual and
informative, with a focus on Lily's personality and preferences.

CURRENT SENTENCE
She likes to play fetch a lot and it's a good way to


AcrosticGeneratorInputs

acrostic_phrase: LILY

original_text:
I know a yellow labrador retriever named Lily.
She is a little over 2 years old, which is pretty much grown up.
She likes to play fetch a lot and it's a good way to tire her out.
She doesn't like to cuddle much, but as she gets older she likes it a bit more.

acrostic_letter_index: 3
rewritten_text:
Lily is the name of the yellow labrador retriever I know.
In just a little over 2 years, Lily has become a fully grown adult.
Lily's favorite activity is playing fetch, which helps wear her out.

        

# test 2

In [10]:
initial_inputs = AcrosticGeneratorInputs(
    original_text = """
The neocortex contains both excitatory (~80%) and inhibitory (~20%) neurons, named for their effect on other neurons.
The human neocortex consists of hundreds of different types of cells.
The structure of the neocortex is relatively uniform (hence the alternative names "iso-" and "homotypic" cortex), consisting of six horizontal layers segregated principally by cell type and neuronal connections.
However, there are many exceptions to this uniformity; for example, layer IV is small or missing in the primary motor cortex. 
There is some canonical circuitry within the cortex; for example, pyramidal neurons in the upper layers II and III project their axons to other areas of neocortex, while those in the deeper layers V and VI often project out of the cortex, e.g. to the thalamus, brainstem, and spinal cord. 
Neurons in layer IV receive the majority of the synaptic connections from outside the cortex (mostly from thalamus), and themselves make short-range, local connections to other cortical layers.
Thus, layer IV is the main recipient of incoming sensory information and distributes it to the other layers for further processing.
""",
    acrostic_phrase = "LAYERS",
    rewritten_text = "",
    acrostic_letter_index = 0,
    n_attempts=3,
)
results = []
verbose = True
inputs = initial_inputs.copy()

In [11]:
current_round_number = 0
n_regenerations_for_current_round = 0

In [13]:
output = full_chain(inputs=inputs.dict())
current_round_result = AcrosticGenerationRoundResult(
    round_number=current_round_number,
    inputs=inputs.copy(),
    output=output,
)
results.append(current_round_result)

if output['next_action'] == 'accept sentence':
    
    selection_idx = output['evaluator_selection'] - 1
    accepted_sentence = output['generated_sentence_options_list'][selection_idx] + ' '
    inputs.rewritten_text += accepted_sentence
    print(f'Accepted sentence:\n{output["evaluator_selection"]}. {accepted_sentence}')
    inputs.acrostic_letter_index += 1
    n_regenerations_for_current_round = 0
else:
    print(f'{output["next_action"]}')
    n_regenerations_for_current_round += 1
current_round_number += 1


print(f"""
SUMMARY
{output['original_text_summary'].strip()}

CONTEXT
{textwrap.fill(output['original_text_context'])}

CURRENT IDEA
{textwrap.fill(output['current_idea'].strip())}

CURRENT SENTENCE
{textwrap.fill(output['current_original_sentence'].strip())}

OPTIONS
{output['clean_options']}
""")

inputs

Accepted sentence:
1. Looking at the neocortex, we find that it has both excitatory and inhibitory neurons, which are named based on their actions on other neurons. 
('\n'
 'SUMMARY\n'
 '  - The neocortex has two types of neurons: excitatory and inhibitory. - '
 'The\n'
 'human neocortex is composed of many different cell types. - The neocortex '
 'has a\n'
 'relatively uniform structure with six horizontal layers, but there are\n'
 'exceptions such as layer IV being small or missing in the primary motor '
 'cortex.\n'
 '- Pyramidal neurons in upper layers project their axons to other areas of\n'
 'neocortex, while those in deeper layers project out of the cortex to other '
 'parts\n'
 'of the brain and spinal cord. - Neurons in layer IV receive most of their\n'
 'synaptic connections from outside the cortex and distribute incoming '
 'sensory\n'
 'information to the other layers for processing.\n'
 '\n'
 'CONTEXT\n'
 '  The text is likely an excerpt from a scientific article or textbo


AcrosticGeneratorInputs

acrostic_phrase: LAYERS

original_text:
The neocortex contains both excitatory (~80%) and inhibitory (~20%) neurons, named for their effect on other neurons.
The human neocortex consists of hundreds of different types of cells.
The structure of the neocortex is relatively uniform (hence the alternative names "iso-" and "homotypic" cortex), consisting of six horizontal layers segregated principally by cell type and neuronal connections.
However, there are many exceptions to this uniformity; for example, layer IV is small or missing in the primary motor cortex.
There is some canonical circuitry within the cortex; for example, pyramidal neurons in the upper layers II and III project their axons to other areas of neocortex, while those in the deeper layers V and VI often project out of the cortex, e.
g.
to the thalamus, brainstem, and spinal cord.
Neurons in layer IV receive the majority of the synaptic connections from outside the cortex (mostly from thalamus), a

In [15]:
output['original_text_summary']

'\n\n- The neocortex has two types of neurons: excitatory and inhibitory.\n- The human neocortex is composed of many different cell types.\n- The neocortex has a relatively uniform structure with six horizontal layers, but there are exceptions such as layer IV being small or missing in the primary motor cortex.\n- Pyramidal neurons in upper layers project their axons to other areas of neocortex, while those in deeper layers project out of the cortex to other parts of the brain and spinal cord.\n- Neurons in layer IV receive most of their synaptic connections from outside the cortex and distribute incoming sensory information to the other layers for processing.'

In [ ]:

print(f"""
SUMMARY
{output['original_text_summary'].strip()}

CONTEXT
{textwrap.fill(output['original_text_context'])}

CURRENT IDEA
{textwrap.fill(output['current_idea'].strip())}

CURRENT SENTENCE
{textwrap.fill(output['current_original_sentence'].strip())}

OPTIONS
{output['clean_options']}
""")


# test 3

In [12]:
initial_inputs = AcrosticGeneratorInputs(
    original_text = """
Ya'll, I made a little LLM steganography project and I want to tell you about it.
My goal was to get chatGPT to hide an acrostic message in a block of text.
I used langchain, which is a really fun library that makes it easy to compose chains of calls to chatGPT.
It's pretty easy to tell it to write a sentence starting with a specific letter. 
But it proved to be a lot harder to get chatGPT to faithfully reproduce the content and style of the original text.
It's prone to hallucinating extra details or to returning outputs that are strained or awkward.
Chain-of-thought prompting seems pretty key.
I asked the LLM to summarize the content and context of the original text and then used that to build a prompt for getting it to rewrite individual sentences.
I also had it make multiple attempts at writing each sentence, and then asked it to evaluate its results and pick the best one.
This approach feels a bit overwrought to me (and it's super slow) but it was a fun exercise.
These tools are pretty amazing and really easy to pick up - I was able to go from nothing to a prototype in a couple of hours.
In fact I used it to hide a secret message in this very post (sorry).
Code here https://github.com/ANaka/stegosaurus
Gradio demo here https://gradio.app/g/ANaka/stegosaurus
""",
    acrostic_phrase = "YOULOSTTHEGAME",
    n_attempts=10,
)
results = []
verbose = True
inputs = initial_inputs.copy()
current_round_number = 0
n_regenerations_for_current_round = 0

In [ ]:
while inputs.acrostic_letter_index < len(inputs.acrostic_phrase):
    output = full_chain(inputs=inputs.dict())
    current_round_result = AcrosticGenerationRoundResult(
        round_number=current_round_number,
        inputs=inputs.copy(),
        output=output,
    )
    results.append(current_round_result)

    if output['next_action'] == 'accept sentence':
        
        selection_idx = output['evaluator_selection'] - 1
        accepted_sentence = output['generated_sentence_options_list'][selection_idx] + ' '
        inputs.rewritten_text += accepted_sentence
        print(f'Accepted sentence:\n{output["evaluator_selection"]}. {accepted_sentence}')
        inputs.acrostic_letter_index += 1
        n_regenerations_for_current_round = 0
    else:
        print(f'{output["next_action"]}')
        n_regenerations_for_current_round += 1
    current_round_number += 1

    print(f"""
ROUND: {current_round_number}

SUMMARY
{output['original_text_summary'].strip()}

CONTEXT
{textwrap.fill(output['original_text_context'])}

CURRENT SENTENCE
{textwrap.fill(output['current_original_sentence'].strip())}

OPTIONS
{output['clean_options']}

ORIGINAL TEXT
{textwrap.fill(output['original_text'])}
    """)


In [23]:
print(textwrap.fill(inputs.rewritten_text))

You know, I developed a LLM steganography project and I'm excited to
tell you all about it. One of my main objectives was to hide an
acrostic message in a block of text using LLM. Unsurprisingly, I used
langchain, a really fun library that makes it easy to compose chains
of calls to chatGPT. LLM makes it simple to write a sentence beginning
with a particular letter. One of the biggest challenges was getting it
to accurately reproduce the content and style of the original text.
Surprisingly, the LLM is prone to hallucinating extra details. To my
surprise, the technique of chain-of-thought prompting proved to be
quite effective. The LLM was asked to generate a summary of the
original text which was used to construct a prompt for rewriting
individual sentences. Having the LLM generate multiple versions of
each sentence, I asked it to evaluate and choose the most suitable one
with hopes of improving accuracy. Even though it was slow and somewhat
convoluted, I enjoyed testing out this appro

In [16]:
import pandas as pd
pd.DataFrame([r.output for r in results])

,original_text,acrostic_phrase,rewritten_text,acrostic_letter_index,n_attempts,original_text_summary,original_text_context,current_starting_letter,current_original_sentence,generated_sentence_options,generated_sentence_options_list,clean_options,evaluator_output,evaluator_selection,next_action
0,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,,0,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,Y,"\nYa'll, I made a little LLM steganography pro...","1. Yes, I created a small steganography projec...","[Yes, I created a small steganography project ...","1. Yes, I created a small steganography projec...","{""selected_option"": 2}",2,accept sentence
1,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",1,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,O,\nMy goal was to get an LLM to hide an acrosti...,"1. Obviously, hiding an acrostic message in a ...","[Obviously, hiding an acrostic message in a bl...","1. Obviously, hiding an acrostic message in a ...","{""selected_option"": 4}",4,accept sentence
2,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",2,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,U,"\nI used langchain, which is a really fun libr...","1. Ultimately, I utilized langchain, an enjoya...","[Ultimately, I utilized langchain, an enjoyabl...","1. Ultimately, I utilized langchain, an enjoya...","{""selected_option"": 6}",6,accept sentence
3,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",3,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,L,\nIt's pretty easy to tell an LLM to write a s...,1. Letting the LLM write a sentence with a spe...,[Letting the LLM write a sentence with a speci...,1. Letting the LLM write a sentence with a spe...,"{""selected_option"": 3}",3,accept sentence
4,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",4,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,O,\nBut it proved to be a lot harder to get it ...,"1. Obviously, faithfully reproducing the conte...","[Obviously, faithfully reproducing the content...","1. Obviously, faithfully reproducing the conte...","{""selected_option"": 4}",4,accept sentence
5,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",5,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,S,\nVery prone to hallucinating extra details,"1. Sometimes, the LLM hallucinates extra detai...","[Sometimes, the LLM hallucinates extra details...","1. Sometimes, the LLM hallucinates extra detai...","{""selected_option"": 2}",2,accept sentence
6,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",6,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,T,"\nUnsurprisingly, chain-of-thought prompting s...","1. To my surprise, the technique of chain-of-t...","[To my surprise, the technique of chain-of-tho...","1. To my surprise, the technique of chain-of-t...","{""selected_option"": 1}",1,accept sentence
7,"\nYa'll, I made a little LLM steganography pro...",YOULOSTTHEGAME,"You know, I developed a LLM steganography proj...",7,10,\n\n- The author created a steganography proje...,\n\nThe author is a programmer who has develop...,T,\nI asked the LLM to summarize the content and...,\n\n1. To build a prompt for rewriting individ...,[To build a prompt for rewriting individual se...,1. To build a prompt for rewriti